In [1]:
import pandas as pd
from datetime import datetime, timedelta, date
from random import randrange
from datetime import timedelta
import numpy as np

def random_date(start, end):
    delta = (end - start).days
    if delta==0:
        return start
    random_day = randrange(delta)
    return start + timedelta(days=random_day)

In [2]:
aisles = pd.read_csv("aisles.csv")
departments = pd.read_csv("departments.csv")
order_products_prior = pd.read_csv("order_products__prior.csv")
order_products_train = pd.read_csv("order_products__train.csv")
orders = pd.read_csv("orders.csv")
products = pd.read_csv("products.csv")
uscities = pd.read_csv("uscities.csv")
WeatherEvents = pd.read_csv("WeatherEvents_Jan2016-Dec2021.csv")
orders['days_since_prior_order'] = orders['days_since_prior_order'].fillna(0)

In [4]:
sum_days_since_prior = orders[['user_id', 'days_since_prior_order']].groupby(['user_id']).sum()
day_of_last_order = date(2017, 1, 1) - timedelta(days=1.0)
first_day = date(2016, 1, 1)
sum_days_since_prior['first_order_date'] = sum_days_since_prior['days_since_prior_order'].apply(lambda x: day_of_last_order - timedelta(days=x))
sum_days_since_prior['first_order_date'] = sum_days_since_prior['first_order_date'].apply(lambda x: random_date(first_day, x))
sum_days_since_prior = sum_days_since_prior.reset_index()
ddd = WeatherEvents['City'].unique()
overlaping_datasets_cities=ddd[np.isin(ddd, uscities['city'].unique())]
sum_days_since_prior['user_location'] = np.random.choice(overlaping_datasets_cities, size=len(sum_days_since_prior))
orders['days_since_first_order']=orders.groupby(['user_id'])['days_since_prior_order'].cumsum()

In [5]:
sum_days_since_prior

,user_id,days_since_prior_order,first_order_date,user_location
0,1,190.0,2016-05-06,Macon
1,2,228.0,2016-05-08,Terrell
2,3,144.0,2016-04-16,Menominee
3,4,85.0,2016-04-09,Pompano Beach
4,5,46.0,2016-06-27,Erwin
...,...,...,...,...
206204,206205,50.0,2016-05-26,Luverne
206205,206206,249.0,2016-03-14,Pine Bluff
206206,206207,229.0,2016-03-17,Owensboro
206207,206208,361.0,2016-01-03,Shelton


In [6]:
orders = pd.merge(orders, sum_days_since_prior[['user_id', 'first_order_date', 'user_location']], on='user_id')
orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,days_since_first_order,first_order_date,user_location
0,2539329,1,prior,1,2,8,0.0,0.0,2016-05-06,Macon
1,2398795,1,prior,2,3,7,15.0,15.0,2016-05-06,Macon
2,473747,1,prior,3,3,12,21.0,36.0,2016-05-06,Macon
3,2254736,1,prior,4,4,7,29.0,65.0,2016-05-06,Macon
4,431534,1,prior,5,4,15,28.0,93.0,2016-05-06,Macon
...,...,...,...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,5,18,29.0,155.0,2016-04-25,Boulder City
3421079,1854736,206209,prior,11,4,10,30.0,185.0,2016-04-25,Boulder City
3421080,626363,206209,prior,12,1,12,18.0,203.0,2016-04-25,Boulder City
3421081,2977660,206209,prior,13,1,12,7.0,210.0,2016-04-25,Boulder City


In [7]:
orders['order_date'] = orders.apply(lambda x: x['first_order_date'] + timedelta(days=x['days_since_first_order']), axis=1)

In [8]:
orders.to_csv('data/orders2.csv')

In [9]:
print(WeatherEvents['City'].value_counts())

Jacksonville          31112
Columbus              30351
Cleveland             28164
Washington            26587
Greenville            26121
                      ...  
Commerce                 54
Campo                    43
Hanksville               34
Colville                 27
South Padre Island        8
Name: City, Length: 1716, dtype: int64


In [10]:
WeatherEvents

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7479160,W-7577193,Storm,Severe,2021-12-26 18:19:00,2021-12-26 18:29:00,0.00,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0
7479161,W-7577194,Snow,Light,2021-12-26 18:29:00,2021-12-26 18:38:00,0.02,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0
7479162,W-7577195,Snow,Light,2021-12-28 00:53:00,2021-12-28 02:27:00,0.08,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0
7479163,W-7577196,Snow,Moderate,2021-12-28 02:27:00,2021-12-28 03:09:00,0.09,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0
